In [1]:
!pip install bs4

Defaulting to user installation because normal site-packages is not writeable


In [2]:
!pip install lxml

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 10.4 MB/s eta 0:00:00a 0:00:01


In [3]:
!pip install requests

Defaulting to user installation because normal site-packages is not writeable


In [11]:
!pip install selenium

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 9.5 MB/s eta 0:00:00a 0:00:01


In [12]:
from bs4 import BeautifulSoup
import requests
from selenium import webdriver

To generate the right **HEADERS** dictionary for your web scraping use case, follow these steps:

1. **Open Developer Tools in your web browser:** Go to the website you want to scrape and open the Developer Tools. Usually this can be done by right-clicking on the page and selecting "Inspect" or "Inspect Element".
2. **Go to the Network Tab:** In the Developer Tools, find and click on the "Network" tab.
3. **Reload the page:** Refresh the website while the Network tab is open. You will see a list of network requests being made.
4. **Find the main request:** Look for the main request that loads the HTML content of the page. This is usually the first or one of the top requests in the list.
5. **Inspect the Headers:** Click on the main request to see its details. Find the "Headers" tab within the request details.
6. **Copy the User-Agent and Accept-Language:** Look for the "User-Agent" and "Accept-Language" headers and copy their values.
7. **Create the HEADERS dictionary:** In your Python code, create a dictionary like the one in your example and paste the copied header values.

In [21]:
driver = webdriver.Chrome()

driver.get('https://www.amazon.com/Best-Sellers-Clothing-Shoes-Jewelry/zgbs/fashion/ref=zg_bs_nav_fashion_0')
html = driver.page_source
soup = BeautifulSoup(html, 'lxml')

products = soup.find_all('div', class_='gridItemRoot')
print(products)


[]


In [24]:
product = soup.find_all('div')

In [26]:

# Amazon Best Sellers URL
url = "https://www.amazon.com/Best-Sellers-Clothing-Shoes-Jewelry/zgbs/fashion/ref=zg_bs_nav_fashion_0"

# Set up headers to mimic a browser
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Accept-Language": "en-US,en;q=0.9",
}

# Send a GET request
response = requests.get(url, headers=headers)

if response.status_code == 200:
    soup = BeautifulSoup(response.content, "html.parser")
    
    # Find all item links on the page
    links = []
    for a_tag in soup.find_all("div", class_="gridItemRoot"):
        href = a_tag.get("href")
        if href and href.startswith("/dp/"):
            full_link = "https://www.amazon.com" + href
            links.append(full_link)

    # Remove duplicates
    unique_links = list(set(links))

    # Print the links
    print("Found", len(unique_links), "links:")
    for link in unique_links:
        print(link)
else:
    print("Failed to fetch the page. Status code:", response.status_code)


Found 0 links:


In [5]:
def main(URL):
    # opening our output file in append mode
    File = open("out.csv", "a")

    # specifying user agent, You can use other user agents
    # available on the internet
    HEADERS = {
        'User-Agent':'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/44.0.2403.157 Safari/537.36',
        'Accept-Language': 'en-US, en;q=0.5'
    }

    # Making the HTTP Request
    webpage = requests.get(URL, headers=HEADERS)

    # Creating the Soup Object containing all data
    soup = BeautifulSoup(webpage.content, "lxml")

    # retrieving product title
    try:
        # Outer Tag Object
        title = soup.find("span", attrs={
            'id': 'productTitle'
        })

        # Inner NavigableString Object
        title_value = title.string

        # Title as a string value
        title_string = title_value.strip().replace(',', '')

    except AttributeError:
        title_string = "NA"
    print("product Title = ", title_string)

    # saving the title in the file
    File.write(f"{title_string},")

    # retrieving price
    try:
        price = soup.find("span", attrs={
            'id': 'priceblock_ourprice'
        }).string.strip().replace(',', '')
        # we are omitting unnecessary spaces
        # and commas form our string
    except AttributeError:
        price = "NA"
    print("Products price = ", price)

    # saving
    File.write(f"{price},")

    # retrieving product rating
    try:
        rating = soup.find("i", attrs={
                           'class': 'a-icon a-icon-star a-star-4-5'
        }).string.strip().replace(',', '')

    except AttributeError:

        try:
            rating = soup.find("span", attrs={
                'class': 'a-icon-alt'
            }).string.strip().replace(',', '')
        except:
            rating = "NA"
    print("Overall rating = ", rating)

    File.write(f"{rating},")

    try:
        review_count = soup.find("span", attrs={
            'id': 'acrCustomerReviewText'
        }).string.strip().replace(',', '')

    except AttributeError:
        review_count = "NA"
    print("Total reviews = ", review_count)
    File.write(f"{review_count},")

    # print availablility status
    try:
        available = soup.find("div", attrs={'id': 'availability'})
        available = available.find("span").string.strip().replace(',', '')

    except AttributeError:
        available = "NA"
    print("Availability = ", available)

    # saving the availability and closing the line
    File.write(f"{available},\n")

    # closing the file
    File.close()

In [6]:
if __name__ == '__main__':
    main("https://www.amazon.com/Dremel-Education-Accessories-Professional-Development/dp/B07KZ8XNDT")

product Title =  Dremel - 3D40-FLX-EDU DigiLab 3D40 Flex 3D Printer w/Extra Supplies 30 Lesson Plans Professional Development Course Flexible Build Plate Automated 9-Point Leveling PC & MAC OS Chromebook iPad Compatible
Products price =  NA
Overall rating =  3.7 out of 5 stars
Total reviews =  46 ratings
Availability =  NA


In [8]:
if __name__ == '__main__':
    main("https://www.amazon.com/Original-Removable-Beginner-Friendly-Calibration-9-84%C3%978-3%C3%978-6/dp/B0CKSW74GX/?_encoding=UTF8&pd_rd_w=sHaZz&content-id=amzn1.sym.255b3518-6e7f-495c-8611-30a58648072e%3Aamzn1.symc.a68f4ca3-28dc-4388-a2cf-24672c480d8f&pf_rd_p=255b3518-6e7f-495c-8611-30a58648072e&pf_rd_r=2GQR5F8HV4R3H1T55724&pd_rd_wg=jhZtJ&pd_rd_r=fa1ae611-71fb-486b-a43d-50a4fabe5a29&ref_=pd_hp_d_atf_ci_mcx_mr_ca_hp_atf_d")

product Title =  Original Prusa MK4 3D Printer kit Removable Print Sheets Beginner-Friendly 3D Printer DYI Kit Fun to Assemble Automatic Calibration Filament Sample Included Print Size 9.84×8.3×8.6 in.
Products price =  NA
Overall rating =  4.3 out of 5 stars
Total reviews =  32 ratings
Availability =  Only 1 left in stock (more on the way).
